
This notebook deal with object detection training using MaskRCNN


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
!pip install tensorflow-gpu==1.15

In [ ]:
!pip install keras==2.2.4

In [ ]:
!ls "/content/gdrive/My Drive/Mask_RCNN/"

assets			      Map_Grid_info
build			      Map_Grid_info_no_row_col
dist			      maptables_cfg20201229T1527
grid_info_3_cfg20210211T1123  maptables_cfg20210117T2356
grid_info_3_cfg20210211T1601  maptables_cfg20210118T2215
grid_info_3_cfg20210211T1917  maptables_cfg20210205T2051
grid_info_3_cfg20210212T1003  maptables_cfg20210215T1813
grid_info_cfg20210209T1539    mask_rcnn_coco.h5
grid_info_cfg20210209T1541    mask_rcnn.egg-info
grid_info_cfg20210212T1959    mrcnn
grid_info_cfg20210215T1356    README.md
grid_info_cfg20210331T0825    requirements.txt
images			      samples
indextable		      setup.cfg
LICENSE			      setup.py
MANIFEST.in


In [ ]:
import sys
sys.path.append('/content/gdrive/My Drive/Mask_RCNN/')

In [ ]:
%cd "/content/gdrive/My Drive/Mask_RCNN/"

/content/gdrive/My Drive/Mask_RCNN


In [ ]:
!python3 setup.py install

/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:645: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:645: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:645: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in future versions. Please use the underscore name 'requirements_file' instead
  % (opt, underscore_opt))
running install
running bdist_egg
running egg_info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
writing manif

In [ ]:
# plot one photograph and mask
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
#from matplotlib import pyplot
#from mrcnn.visualize import display_instances
#from mrcnn.utils import extract_bboxes
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
import imgaug

Using TensorFlow backend.


In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()

True

In [ ]:
######################### Prepare Dataset ##################################
category = 'gridinfodataset'
class_names = ['Grid_info']

# class that defines and loads the kangaroo dataset
class GridInfoDataset(Dataset):
    # load the dataset definitions
    def load_dataset(self, dataset_dir, is_train=True):
        # define one class
        #self.add_class("gridinfodataset", 1, "Grid_info")
        for i in range(1,len(class_names)+1):
            self.add_class(category, i, class_names[i-1])
        # define data locations
        if is_train:
            images_dir=dataset_dir+'/train/'
            annotations_dir=dataset_dir+'/train_annots/'
        else:
            images_dir=dataset_dir+'/val/'
            annotations_dir=dataset_dir+'/val_annots/'
        # find all images
        c=0
        for filename in listdir(images_dir):
            # extract image id
            image_id = filename[:-4] # filename is for instance 0001.jpg so
            print("image_id : ",image_id)
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            c=c+1
            # add to dataset
            self.add_image(category, image_id=image_id, path=img_path, annotation=ann_path)
    # extract bounding boxes from an annotation file
    def extract_boxes(self, filename):
        # load and parse the file
        tree = ElementTree.parse(filename)
        # get the root of the document
        root = tree.getroot()
        # extract each bounding box
        boxes = list()
        box_class_name = list()
        no_rows = list()
        no_cols = list()
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
        for name in root.findall('.//name'):
            box_class_name.append(name.text)

        for rows in root.findall('.//no_rows'):
            no_rows.append(rows.text)

        for cols in root.findall('.//no_cols'):
            no_cols.append(cols.text)
        # extract image dimensions

        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        return boxes, width, height,box_class_name,no_rows,no_cols

    # load the masks for an image
    def load_mask(self, image_id):
        # get details of image
        info = self.image_info[image_id]
        # define box file location , xml file
        path = info['annotation']
        # load XML
        boxes, w, h,box_class_name,no_rows,no_cols = self.extract_boxes(path)

        # create one array for all masks, each on a different channel
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        # create masks
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            #row_s , col_s is the starting position (i,j) of box
            #row_e , col_e is the ending position (i,j) of box
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index(box_class_name[i]))


        return masks, asarray(class_ids, dtype='int32'),asarray(no_rows, dtype='int32'),asarray(no_cols, dtype='int32')

    # load an image reference
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']
################################ Start Training ##############################

#set paths according to your data location
dataset_path ='/content/gdrive/My Drive/Mask_RCNN/Map_Grid_info'
model_directory_path='/content/gdrive/My Drive/Mask_RCNN/Map_Grid_info_no_row_col'
pretrained_weights_path='/content/gdrive/My Drive/Mask_RCNN/Map_Grid_info_no_row_col/grid_info_cfg20210420T1612/mask_rcnn_grid_info_cfg_0137.h5'
#pretrained_weights_path='/content/gdrive/My Drive/Mask_RCNN/mask_rcnn_coco.h5'
class GridInfoConfig(Config):
    # Give the configuration a recognizable name
    NAME = "Grid_info_cfg"
    # Number of classes (background + kangaroo)
    NUM_CLASSES = 1 + len(class_names)
    # Number of training steps per epoch
    STEPS_PER_EPOCH = 228
    IMAGES_PER_GPU = 1
    BATCH_SIZE = 1
    VALIDATION_STEPS = 24

# prepare config
config = GridInfoConfig()

# prepare train set
print("############################# TRAIN #########################")
train_set = GridInfoDataset()
train_set.load_dataset(dataset_path , is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# prepare test/val set
print("############################# TEST #########################")
test_set = GridInfoDataset()
test_set.load_dataset(dataset_path , is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
# prepare config
config = GridInfoConfig()
config.display()
# define the model
model = MaskRCNN(mode='training', model_dir=model_directory_path , config=config)
# load weights (mscoco) and exclude the output layers   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"]
#model.load_weights(pretrained_weights_path, by_name=True,exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
model.load_weights(pretrained_weights_path, by_name=True,exclude=["mrcnn_row_conv1", "mrcnn_col_conv1",  "mrcnn_row_bn1", "mrcnn_col_bn1", "mrcnn_row_conv2", "mrcnn_col_conv2",  "mrcnn_row_bn2", "mrcnn_col_bn2","mrcnn_no_row","mrcnn_no_col"])
#model.load_weights(pretrained_weights_path, by_name=True)
# train weights (output layers or 'heads')
#aug = imgaug.augmenters.Sometimes(0.75,imgaug.augmenters.OneOf(
#                                            [
#                                            imgaug.augmenters.GaussianBlur(sigma=(0.0, 5.0)),
#                                            imgaug.augmenters.Affine(rotate=(-45, 45)),
#                                            imgaug.augmenters.Affine(rotate=(-30, 30)),
#                                            imgaug.augmenters.Affine(rotate=(-60, 60)),
#                                            imgaug.augmenters.Affine(rotate=(-15, 15)),
#                                            imgaug.augmenters.Affine(scale=(0.5, 1.5))
#                                             ]
#                                        )
#                                   )

aug = imgaug.augmenters.Sometimes(0.5,imgaug.augmenters.OneOf(
                                            [
                                            imgaug.augmenters.GaussianBlur(sigma=(0.0, 5.0)),
                                            # imgaug.augmenters.Affine(rotate=(-45, 45)),
                                            imgaug.augmenters.Affine(scale=(0.5, 1.5))
                                             ]
                                        )
                                   )
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=1000, layers='heads',augmentation = aug)

#model.keras_model.summary()
# imgaug.Sometimes(5/6,aug.OneOf(
#                                             [
#                                             imgaug.augmenters.Fliplr(1),
#                                             imgaug.augmenters.Flipud(1),
#                                             imgaug.augmenters.Affine(rotate=(-45, 45)),
#                                             imgaug.augmenters.Affine(rotate=(-90, 90)),
#                                             imgaug.augmenters.Affine(scale=(0.5, 1.5))
#                                              ]
#                                         )
#                                    )


############################# TRAIN #########################
image_id :  massavi_134
image_id :  massavi_2334
image_id :  massavi_1324
image_id :  massavi_1560
image_id :  massavi_758
image_id :  massavi_757
image_id :  massavi_1640
image_id :  massavi_269
image_id :  massavi_952
image_id :  massavi_1992
image_id :  massavi_233
image_id :  massavi_2111
image_id :  massavi_1754
image_id :  massavi_2330
image_id :  massavi_1702
image_id :  massavi_160
image_id :  massavi_2047
image_id :  massavi_10
image_id :  massavi_1624
image_id :  massavi_13
image_id :  massavi_1426
image_id :  massavi_722
image_id :  massavi_882
image_id :  massavi_1290
image_id :  massavi_642
image_id :  massavi_701
image_id :  massavi_1746
image_id :  massavi_621
image_id :  massavi_2333
image_id :  massavi_142
image_id :  massavi_1857
image_id :  massavi_2286
image_id :  massavi_946
image_id :  massavi_838
image_id :  massavi_2311
image_id :  massavi_1773
image_id :  massavi_1468
image_id :  massavi_1431
image_i

/usr/local/lib/python3.7/dist-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'




Epoch 88/1000
228/228 [==============================] - 202s 887ms/step - loss: 1.1834 - rpn_class_loss: 0.0268 - rpn_bbox_loss: 0.3142 - mrcnn_class_loss: 0.0305 - mrcnn_bbox_loss: 0.0926 - mrcnn_mask_loss: 0.3689 - mrcnn_no_row_loss: 0.2061 - mrcnn_no_col_loss: 0.1444 - val_loss: 1.3756 - val_rpn_class_loss: 4.1913e-04 - val_rpn_bbox_loss: 0.1545 - val_mrcnn_class_loss: 0.0435 - val_mrcnn_bbox_loss: 0.2155 - val_mrcnn_mask_loss: 0.7346 - val_mrcnn_no_row_loss: 0.1226 - val_mrcnn_no_col_loss: 0.1045

Epoch 89/1000
228/228 [==============================] - 105s 459ms/step - loss: 1.1734 - rpn_class_loss: 0.0327 - rpn_bbox_loss: 0.3159 - mrcnn_class_loss: 0.0663 - mrcnn_bbox_loss: 0.1143 - mrcnn_mask_loss: 0.3449 - mrcnn_no_row_loss: 0.1621 - mrcnn_no_col_loss: 0.1374 - val_loss: 1.2568 - val_rpn_class_loss: 0.0054 - val_rpn_bbox_loss: 0.1185 - val_mrcnn_class_loss: 0.0739 - val_mrcnn_bbox_loss: 0.1918 - val_mrcnn_mask_loss: 0.7081 - val_mrcnn_no_row_loss: 0.0541 - val_mrcnn_no_col_